**This notebook is an exercise in the [Advanced SQL](https://www.kaggle.com/learn/advanced-sql) course.  You can reference the tutorial at [this link](https://www.kaggle.com/alexisbcook/analytic-functions).**

---


# Introduction

Here, you'll use window functions to answer questions about the [Chicago Taxi Trips](https://www.kaggle.com/chicago/chicago-taxi-trips-bq) dataset.

Before you get started, run the code cell below to set everything up.

In [2]:
# Get most recent checking code
!pip install -U -t /kaggle/working/ git+https://github.com/Kaggle/learntools.git
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql_advanced.ex2 import *
print("Setup Complete")

  Cloning https://github.com/Kaggle/learntools.git to /tmp/pip-req-build-nxm8apa4
  Running command git clone --filter=blob:none --quiet https://github.com/Kaggle/learntools.git /tmp/pip-req-build-nxm8apa4
  Resolved https://github.com/Kaggle/learntools.git to commit ab3f19a77b8a92c7337c9fd15d84602e7320a077
  Preparing metadata (setup.py) ... done
  Created wheel for learntools: filename=learntools-0.3.4-py3-none-any.whl size=268981 sha256=5f2f3b48996ed25a62450da22dd7a69f6354ac2b3be59170d3b7322ed589e9e3
  Stored in directory: /tmp/pip-ephem-wheel-cache-m_8j_wvc/wheels/2f/6c/3c/aa9f50cfb5a862157cb4c7a5b34881828cf45404698255dced
Successfully built learntools
Using Kaggle's public dataset BigQuery integration.
Setup Complete


The following code cell fetches the `taxi_trips` table from the `chicago_taxi_trips` dataset. We also preview the first five rows of the table.  You'll use the table to answer the questions below.

In [4]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "chicago_taxi_trips" dataset
dataset_ref = client.dataset("chicago_taxi_trips", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "taxi_trips" table
table_ref = dataset_ref.table("taxi_trips")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

Using Kaggle's public dataset BigQuery integration.


,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,771053a9bad80a7c1c0d86c5fb785f21207e59c7,d0cd60cfd2405562733ddd1ea3aba9be5ce4f20bc622c5...,2019-07-01 13:45:00+00:00,2019-07-01 14:00:00+00:00,753,2.28,1.703133e+10,1.703108e+10,33,8,...,1.0,11.25,Cash,Sun Taxi,41.859350,-87.617358,POINT (-87.6173580061 41.859349715),41.892508,-87.626215,POINT (-87.6262149064 41.8925077809)
1,735f7dac03f027bb51ea12f8d6fabeb2ae9dc5b9,044aa88179fa43c628fe5a956567224cf0b1b1ad5994f3...,2019-07-28 11:45:00+00:00,2019-07-28 11:45:00+00:00,792,2.99,1.703108e+10,1.703133e+10,8,33,...,0.5,15.00,Credit Card,Chicago Carriage Cab Corp,41.892042,-87.631864,POINT (-87.6318639497 41.8920421365),41.859350,-87.617358,POINT (-87.6173580061 41.859349715)
2,739fe5c84ae4e748193af9a9285ff35f27c9bac3,83946c2d33e0cac34a4d8a03a8bdca905808e852e16727...,2019-07-28 00:45:00+00:00,2019-07-28 01:15:00+00:00,1860,17.80,1.703198e+10,1.703108e+10,76,8,...,4.0,49.00,Cash,Star North Management LLC,41.979071,-87.903040,POINT (-87.9030396611 41.9790708201),41.899156,-87.626211,POINT (-87.6262105324 41.8991556134)
3,7417d5ef4423e3d3fd1e1be5cc925e26bb2a6c43,a4d3ebdb843bb0ff2eeca8acc59adae88cf333e9efb25d...,2019-07-02 11:45:00+00:00,2019-07-02 12:00:00+00:00,526,1.18,1.703184e+10,1.703183e+10,32,28,...,0.0,7.25,Cash,Chicago Carriage Cab Corp,41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),41.885281,-87.657233,POINT (-87.6572331997 41.8852813201)
4,763d46c61d3ee7799291f13b704177ca599e067a,dae02605eae3567faac3548acc8480606cad8d08fac7bc...,2019-07-13 22:30:00+00:00,2019-07-13 22:45:00+00:00,402,0.99,NaN,NaN,7,6,...,0.0,6.00,Cash,Flash Cab,41.922686,-87.649489,POINT (-87.6494887289 41.9226862843),41.944227,-87.655998,POINT (-87.6559981815 41.9442266014)


ROWS BETWEEN 3 PRECEDING AND 3 FOLLOWING# Exercises

### 1) How can you predict the demand for taxis?

Say you work for a taxi company, and you're interested in predicting the demand for taxis.  Towards this goal, you'd like to create a plot that shows a rolling average of the daily number of taxi trips.  Amend the (partial) query below to return a DataFrame with two columns:
- `trip_date` - contains one entry for each date from January 1, 2016, to March 31, 2016.
- `avg_num_trips` - shows the average number of daily trips, calculated over a window including the value for the current date, along with the values for the preceding 3 days and the following 3 days, as long as the days fit within the three-month time frame.  For instance, when calculating the value in this column for January 3, 2016, the window will include the number of trips for the preceding 2 days, the current date, and the following 3 days.

This query is partially completed for you, and you need only write the part that calculates the `avg_num_trips` column.  Note that this query uses a common table expression (CTE); if you need to review how to use CTEs, you're encouraged to check out [this tutorial](https://www.kaggle.com/dansbecker/as-with) in the [Intro to SQL](https://www.kaggle.com/learn/intro-to-sql) course.

In [8]:
# Fill in the blank below
avg_num_trips_query = """
                      WITH trips_by_day AS
                      (
                      SELECT DATE(trip_start_timestamp) AS trip_date,
                          COUNT(*) as num_trips
                      FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                      WHERE trip_start_timestamp > '2016-01-01' AND trip_start_timestamp < '2016-04-01'
                      GROUP BY trip_date
                      ORDER BY trip_date
                      )
                      SELECT trip_date,
                          num_trips,
                          AVG(num_trips)
                          OVER (
                               ORDER BY trip_date
                               ROWS BETWEEN 3 PRECEDING AND 3 FOLLOWING
                               ) AS avg_num_trips
                      FROM trips_by_day
                      ORDER BY trip_date
                      """

# Check your answer
q_1.check()

,trip_date,num_trips,avg_num_trips
0,2016-01-01,96216,73067.250000
1,2016-01-02,69000,72801.000000
2,2016-01-03,59286,73287.833333
3,2016-01-04,67767,74260.571429
4,2016-01-05,71736,73767.285714


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [ ]:
# Lines below will give you a hint or solution code
#q_1.hint()
#q_1.solution()

### 2) Can you separate and order trips by community area?

The query below returns a DataFrame with three columns from the table: `pickup_community_area`, `trip_start_timestamp`, and `trip_end_timestamp`.  

Amend the query to return an additional column called `trip_number` which shows the order in which the trips were taken from their respective community areas.  So, the first trip of the day originating from community area 1 should receive a value of 1; the second trip of the day from the same area should receive a value of 2.  Likewise, the first trip of the day from community area 2 should receive a value of 1, and so on.

Note that there are many numbering functions that can be used to solve this problem (depending on how you want to deal with trips that started at the same time from the same community area); to answer this question, please use the **RANK()** function.

In [16]:
# Amend the query below
trip_number_query = """
                    SELECT pickup_community_area,
                        trip_start_timestamp,
                        trip_end_timestamp,
                        RANK() 
                            OVER(
                                PARTITION BY pickup_community_area
                                ORDER BY trip_start_timestamp
                            ) AS trip_number
                    FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                    WHERE DATE(trip_start_timestamp) = '2013-10-03'
                    """

# Check your answer
q_2.check()

,pickup_community_area,trip_start_timestamp,trip_end_timestamp,trip_number
0,18.0,2013-10-03 01:45:00+00:00,2013-10-03 01:45:00+00:00,1
1,18.0,2013-10-03 08:15:00+00:00,2013-10-03 08:15:00+00:00,2
2,18.0,2013-10-03 11:45:00+00:00,2013-10-03 12:00:00+00:00,3
3,18.0,2013-10-03 12:00:00+00:00,2013-10-03 12:00:00+00:00,4
4,10.0,2013-10-03 02:00:00+00:00,2013-10-03 02:00:00+00:00,1


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [ ]:
# Lines below will give you a hint or solution code
#q_2.hint()
#q_2.solution()

### 3) How much time elapses between trips?

The (partial) query in the code cell below shows, for each trip in the selected time frame, the corresponding `taxi_id`, `trip_start_timestamp`, and `trip_end_timestamp`. 

Your task in this exercise is to edit the query to include an additional `prev_break` column that shows the length of the break (in minutes) that the driver had before each trip started (this corresponds to the time between `trip_start_timestamp` of the current trip and `trip_end_timestamp` of the previous trip).  Partition the calculation by `taxi_id`, and order the results within each partition by `trip_start_timestamp`.

Some sample results are shown below, where all rows correspond to the same driver (or `taxi_id`).  Take the time now to make sure that the values in the `prev_break` column make sense to you!

![first_commands](https://storage.googleapis.com/kaggle-media/learn/images/qjvQzg8.png)

Note that the first trip of the day for each driver should have a value of **NaN** (not a number) in the `prev_break` column.

In [26]:
# Fill in the blanks below
break_time_query = """
                   SELECT taxi_id,
                       trip_start_timestamp,
                       trip_end_timestamp,
                       TIMESTAMP_DIFF(
                           trip_start_timestamp, 
                           LAG(trip_end_timestamp) 
                               OVER (
                                    PARTITION BY taxi_id 
                                    ORDER BY trip_start_timestamp), 
                           MINUTE) as prev_break
                   FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                   WHERE DATE(trip_start_timestamp) = '2013-10-03'
                   """

# Check your answer
q_3.check()

,taxi_id,trip_start_timestamp,trip_end_timestamp,prev_break
0,070518ef3fd0616a6819fce1a5b13035c177230f85ed64...,2013-10-03 05:15:00+00:00,2013-10-03 05:30:00+00:00,285.0
1,070518ef3fd0616a6819fce1a5b13035c177230f85ed64...,2013-10-03 22:45:00+00:00,2013-10-03 23:15:00+00:00,900.0
2,07465cb4e0f3794b4d096c8cdbf8be2a7b31851db743de...,2013-10-03 18:45:00+00:00,2013-10-03 19:00:00+00:00,210.0
3,093e9e4c05ea53bf75c51763839d5f5bd5d1785c11ee5e...,2013-10-03 17:30:00+00:00,2013-10-03 17:30:00+00:00,180.0
4,0c8698dd0e6d968d193771b14459e0ad120684b5fb14d0...,2013-10-03 08:30:00+00:00,2013-10-03 08:15:00+00:00,330.0


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [ ]:
# Lines below will give you a hint or solution code
#q_3.hint()
#q_3.solution()

# Keep going

Move on to learn how to query **[nested and repeated data](https://www.kaggle.com/alexisbcook/nested-and-repeated-data)**.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/advanced-sql/discussion) to chat with other learners.*